In [ ]:
%reset -sf

In [ ]:
# LOAD THIS BEFORE START, run rmdl

#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# import numpy as np
import scipy
import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
abc_map = {c:i for i,c in enumerate(abc)}
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [ ]:
with open("/Users/htong/Downloads/input.txt") as f:
    srr = f.read().strip().split("\n")

In [ ]:
srr = """
jqt: rhn xhk nvd
rsh: frs pzl lsr
xhk: hfx
cmg: qnr nvd lhk bvb
rhn: xhk bvb hfx
bvb: xhk hfx
pzl: lsr hfx nvd
qnr: nvd
ntq: jqt hfx bvb xhk
nvd: lhk
lsr: lhk
rzs: qnr cmg lsr rsh
frs: qnr lhk lsr
""".strip().split("\n")

In [ ]:
g = defaultdict(list)

In [ ]:
edges = []
nodes = set()

In [ ]:
for row in srr:
    a,b = row.split(": ")
    brr = b.split()
    for b in brr:
        edges.append((a,b))
        nodes.add(a)
        nodes.add(b)

In [ ]:
# edges.remove(("qnz", "gzr"))
edges.remove(("gzr", "qnz"))

edges.remove(("xgs", "lmj"))
# edges.remove(("lmj", "xgs"))

# edges.remove(("pgz", "hgk"))
edges.remove(("hgk", "pgz"))

In [ ]:
ds = DisjointSet()
for a,b in edges:
    ds.union(a,b)

arr = [ds.find(x) for x in nodes]
if len(set(arr)) == 2:
    print(Counter(arr))

In [ ]:
732 * 729

In [ ]:
class DisjointSet:
    # leetcode.com/problems/accounts-merge/
    def __init__(self, parent={}):
        if not parent:
            parent = {}
        self.parent = parent

    def find(self, item):
        if item not in self.parent:
            self.parent[item] = item
            return item
        elif self.parent[item] == item:
            return item
        else:
            res = self.find(self.parent[item])
            self.parent[item] = res
            return res

    def union(self, set1, set2):
        root1 = self.find(set1)
        root2 = self.find(set2)
        self.parent[root1] = root2

In [ ]:
print(len(edges))

In [ ]:
random.shuffle(edges)

In [ ]:
for comb in itertools.combinations(edges, len(edges) - 3):
    ds = DisjointSet()
    for a,b in comb:
        ds.union(a,b)
    
    arr = [ds.find(x) for x in nodes]
    if len(set(arr)) == 2:
        print(Counter(arr))
        break

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

plt.figure(figsize=(14,14))

# Create a graph object
G = nx.Graph()

# Add nodes to the graph
for node in nodes:
    G.add_node(node)

# Add edges to the graph (also adds nodes if they don't exist)
for a,b in edges:
    G.add_edge(a, b)

# Draw the graph
nx.draw(G, with_labels=True, font_weight='bold')

# Show the plot
plt.show()

In [ ]:
edges

In [ ]:
hgk gzr


In [ ]:
import networkx


In [ ]:
g = defaultdict(list)

for a,b in edges:
    g[a].append(b)
    g[b].append(a)


In [ ]:
for k,v in g.items():
    print(k, v)

In [ ]:
for a,b in edges:
    print(a,b)